In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import plot_model
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from skimage import io, color
import warnings
import cv2

In [ ]:
#Pre_Process_For Direct SEM Images


In [ ]:
reference_img = 'ref/ref1_b.jpg'
reference_img_tmp = 'tmp/ref.png'
size = (400,400)
rgb = io.imread(reference_img)
resized_image = resize(rgb, size)
rescaled_image = 255 * resized_image
final_image = rescaled_image.astype(np.uint8)
io.imsave(reference_img_tmp,final_image)


In [ ]:
dir_sem = 'sem/' 
sem_input = 'AI-01.jpg'
sem_input_tmp = 'tmp/sem.png'
size = (400,400)
rgb = io.imread(dir_sem+sem_input)
resized_image = resize(rgb, size)
rescaled_image = 255 * resized_image
final_image = rescaled_image.astype(np.uint8)
io.imsave(sem_input_tmp,final_image)

In [ ]:
fig = plt.figure()

a=fig.add_subplot(1,2,1)
img_rslt=mpimg.imread(reference_img_tmp)
imgplot= plt.imshow(img_rslt)
a.set_title('Reference Image')
a.axis('off')
a=fig.add_subplot(1,2,2)
img_gray=mpimg.imread(sem_input_tmp)
imgplot= plt.imshow(img_gray, cmap='gray')
a.set_title('SEM Image')

a.axis('off')
plt.show()

In [ ]:
# Get images

reference_image = img_to_array(load_img(reference_img_tmp))
reference_image = np.array(reference_image, dtype=float)
sem_image = img_to_array(load_img(sem_input_tmp))
sem_image = np.array(sem_image, dtype=float)


In [ ]:
X = rgb2lab(1.0/255*reference_image)[:,:,0]
Y = rgb2lab(1.0/255*reference_image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)
## TO be Draw
X_sem = rgb2lab(1.0/255*sem_image)[:,:,0]
#X_style = style_image.reshape((image.shape[0] * image.shape[1], 3))
X_sem = X_sem.reshape(1, 400, 400, 1)



In [ ]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')
#plot_model(model, to_file='model.png')

In [ ]:
history = model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=500)

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
color_dict = {}
color_dict_max = {}
color_dict_max_ab = {}
def same_grey_same_colorize(AB, L):
    
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            l = L[i][j]
            ab = list(map(lambda x:int(x), AB[i][j] + 128))
            if l not in color_dict:
                color_dict[l] = [[0]*256]*256
                color_dict_max[l] = 0
            color_dict[l][ab[0]][ab[1]] += 1
            if color_dict[l][ab[0]][ab[1]] > color_dict_max[l]:
                color_dict_max[l] = color_dict[l][ab[0]][ab[1]]
                color_dict_max_ab[l] = AB[i][j]
                
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            l = L[i][j]
            AB[i][j] = color_dict_max_ab[l]
    return AB

In [ ]:
#debug code
#l = np.array([[1,3,4,1],[3,4,1,3]])
#ab = np.array([[5,7,8,5],[8,9,6,7]])
#same_grey_same_colorize(ab, l)

In [ ]:
print(model.evaluate(X_sem, Y, batch_size=1))

output = model.predict(X_sem)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
L = X_sem[0][:,:,0]
AB = output[0]
cur[:,:,0] = L
cur[:,:,1:] = same_grey_same_colorize(AB, L)
imsave("results_nst/"+sem_input, lab2rgb(cur))

In [ ]:
fig = plt.figure()

a=fig.add_subplot(1,2,1)
img_rslt=mpimg.imread('results_nst/'+sem_input)
imgplot= plt.imshow(img_rslt)
a.set_title('Result')
a.axis('off')
a=fig.add_subplot(1,2,2)
imgplot= plt.imshow(sem_image/255,cmap='gray')
a.set_title('Gray')

a.axis('off')
plt.show()